In [ ]:
from CBR.casebase import *

small_sets = False
size = 300 if small_sets else -1

# A link to the (processed) csv file. 
# Feel free to try some different values. 
# csv = "data/mushrooms.csv"
csv = "data/admission.csv"
# csv = "data/tort.csv"
# csv = "data/welfare.csv"

# Load the case base with logistic regression orders.
CB = casebase(
    csv,
    verb=True, # The verbose mode prints the dimension order information. 
    method='logreg',
    size=size, # Truncates the size of the resulting case base to 'size', 
               # or uses the full size of the csv if size == -1. 
    shuffle=True,
    )

# 'analyze' is a conenience functions that bundles the other analysis functions,
# check the casebase.py file to see (or change) what it does.
CB.analyze()

In [ ]:
# Load the first two cases in the case base in variables a and b. 
a = CB[1]
b = CB[7]

# Print a comparison between the cases a and b.
# This shows for each dimension d the values a(d) and b(d),
# the relation between them (so a(d) < b(d), or a(d) > b(d), etc).
CB.compare(a, b)

# Dimensions on which b is not better than a are called 'relevant differences.
# These can be computer with the .diff function of cases.
# Note that these are also indicated by the 'compare' function.
relevant_differences = a.diff(b)
print(list(relevant_differences))

In [ ]:
# As we know, if c has outcome s then c forces the outcome of d for s iff D(c, d) = {}. 
# This means we can use the .diff function to check whether a CB forces the outcome of a case.
a = CB[3] # change to CB[0] for an example of a case that was not forced. 
force = False
for b in CB:
    if not a == b:
        if list(b.diff(a)) == []:
            force = True
            print("The outcome of the input case a was forced by a case b in the case base:")
            CB.compare(b, a)
            break

if not force:
    print("The outcome of case a was not forced by CB.")

In [ ]:
# Compute the size of the case base, and make an ~80% split. 
CB_size = len(list(CB.inds))
split = CB_size - (CB_size // 5)
CB_train = CB[:split]
CB_test = CB[split:]
print(f"Total size of CB: {CB_size}")
print(f"Taking first {split} for train, and {CB_size - split} for test.")

# Compute whether the first case of the test split is forced by the train split. 
a = CB_train[0]
for b in CB_train:
    if not a == b:
        if list(b.diff(a)) == []:
            force = True
            print("The outcome of the input case a was forced by a case b in the training case base:")
            CB.compare(b, a)
            # De 'a' en 'b' zijn hier omgedraaid t.o.v. de output, beetje onhandig maar goed... 
            break